# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
vacation_df=pd.read_csv('../output_data/cities.csv')
vacation_df.count()

City          537
Lat           537
Lng           537
Max Temp      537
Humidity      537
Cloudiness    537
Wind Speed    537
Country       534
Date          537
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

humidity= vacation_df["Humidity"]

In [49]:
# Plot Heatmap
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
better_vacation_df= vacation_df.loc[(vacation_df["Max Temp"]>70 ) & (vacation_df["Max Temp"]< 80) & (vacation_df["Wind Speed"]<= 10) 
                                    & (vacation_df["Cloudiness"]==0)].dropna()
better_vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,sawakin,19.1059,37.3321,79.27,76,0,5.84,SD,1635037159
136,atar,20.5169,-13.0499,78.94,18,0,7.70,MR,1635037183
155,geraldton,-28.7667,114.6000,70.21,56,0,1.14,AU,1635037043
206,great bend,38.3645,-98.7648,71.10,71,0,6.49,US,1635037209
230,san patricio,28.0170,-97.5169,77.36,69,0,9.48,US,1635037222
318,saint-pierre,-21.3393,55.4781,71.85,77,0,4.61,RE,1635037229
405,buraydah,26.3260,43.9750,74.03,19,0,10.00,SA,1635037286
408,aswan,24.0934,32.9070,70.90,43,0,6.91,EG,1635037186
458,mecca,21.4267,39.8261,78.96,60,0,1.70,SA,1635037202
477,riyadh,24.6877,46.7219,77.18,24,0,9.53,SA,1635037310


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:

hotel_df=better_vacation_df
hotel_df["Hotel Name"]= ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,sawakin,19.1059,37.3321,79.27,76,0,5.84,SD,1635037159,
136,atar,20.5169,-13.0499,78.94,18,0,7.70,MR,1635037183,
155,geraldton,-28.7667,114.6000,70.21,56,0,1.14,AU,1635037043,
206,great bend,38.3645,-98.7648,71.10,71,0,6.49,US,1635037209,
230,san patricio,28.0170,-97.5169,77.36,69,0,9.48,US,1635037222,
318,saint-pierre,-21.3393,55.4781,71.85,77,0,4.61,RE,1635037229,
405,buraydah,26.3260,43.9750,74.03,19,0,10.00,SA,1635037286,
408,aswan,24.0934,32.9070,70.90,43,0,6.91,EG,1635037186,
458,mecca,21.4267,39.8261,78.96,60,0,1.70,SA,1635037202,
477,riyadh,24.6877,46.7219,77.18,24,0,9.53,SA,1635037310,


In [52]:
import pprint
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
#     "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
       
            # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

            # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"

        
#     try:
        # make request and print url
        response = requests.get(base_url, params=params).json()
#         pprint.pprint(response)
#         break

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
#     try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,sawakin,19.1059,37.3321,79.27,76,0,5.84,SD,1635037159,فندق الدويم
136,atar,20.5169,-13.0499,78.94,18,0,7.70,MR,1635037183,Oumou Elghoura
155,geraldton,-28.7667,114.6000,70.21,56,0,1.14,AU,1635037043,Broadwater Mariner Resort
206,great bend,38.3645,-98.7648,71.10,71,0,6.49,US,1635037209,Best Western Angus Inn
230,san patricio,28.0170,-97.5169,77.36,69,0,9.48,US,1635037222,"Motel 6 Sinton, TX"
318,saint-pierre,-21.3393,55.4781,71.85,77,0,4.61,RE,1635037229,Lindsey Hôtel
405,buraydah,26.3260,43.9750,74.03,19,0,10.00,SA,1635037286,فندق راحة السلام
408,aswan,24.0934,32.9070,70.90,43,0,6.91,EG,1635037186,Nuba Nile
458,mecca,21.4267,39.8261,78.96,60,0,1.70,SA,1635037202,"Makkah Clock Royal Tower, A Fairmont Hotel"
477,riyadh,24.6877,46.7219,77.18,24,0,9.53,SA,1635037310,"InterContinental Riyadh, an IHG Hotel"


In [53]:
# for index, row in hotel_df.iterrows():
#     print(row)
#     break


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# hotel_info

In [55]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content= hotel_info)

# Display figure
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))